In [ ]:
import re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def clean_tweet(text):
    text = re.sub(r'http\S+', '', text)        
    text = re.sub(r'@\w+', '', text)           
    text = re.sub(r'#\w+', '', text)           
    text = re.sub(r'[^\w\s]', '', text)        
    text = re.sub(r'\d+', '', text)            
    text = text.strip()
    return text


* import & Cleaning Training Data 

In [ ]:
n_tweets=pd.read_csv('train_Arabic_tweets_negative_20190413.tsv',sep='\t',header=None)
p_tweets=pd.read_csv('train_Arabic_tweets_positive_20190413.tsv',sep='\t',header=None)

In [ ]:
# Concate both
df_train = pd.concat([n_tweets, p_tweets], ignore_index=True)
df_train.columns = ['label', 'tweet']

df_train.head()

,label,tweet
0,neg,اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم...
1,neg,توقعت اذا جات داريا بشوفهم كاملين بس لي للحين ...
2,neg,#الاهلي_الهلال اكتب توقعك لنتيجة لقاء الهلال و...
3,neg,نعمة المضادات الحيوية . تضع قطرة💧مضاد بنسلين ع...
4,neg,الدودو جايه تكمل علي 💔


In [ ]:
x_train = df_train['tweet']
y_train = df_train['label']

In [ ]:
x_train=x_train.apply(clean_tweet)
x_train.head()

0    اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم...
1    توقعت اذا جات داريا بشوفهم كاملين بس لي للحين ...
2    اكتب توقعك لنتيجة لقاء الهلال والاهلي تحت التا...
3    نعمة المضادات الحيوية  تضع قطرةمضاد بنسلين على...
4                                 الدودو جايه تكمل علي
Name: tweet, dtype: object

In [ ]:
#check balance or no
y_train.value_counts()

label
pos    22761
neg    22514
Name: count, dtype: int64

* import & Cleaning Testing Data 

In [ ]:
n_tweetss=pd.read_csv('test_Arabic_tweets_negative_20190413.tsv',sep='\t',header=None)
p_tweetss=pd.read_csv('test_Arabic_tweets_positive_20190413.tsv',sep='\t',header=None)

In [ ]:
# Concate both
df_test = pd.concat([n_tweetss, p_tweetss], ignore_index=True)
df_test.columns = ['label', 'tweet']

In [ ]:
x_test = df_test['tweet']
y_test = df_test['label']

In [ ]:
x_test=x_test.apply(clean_tweet)
x_test.head()

0                    حتى الايتونز خربتوه مو صاحين انتو
1    واحد تبع النظام السوري يقول أن المخابرات السور...
2    الى متى التعامل السئ للخادمات وعدم احترامهم وك...
3                                     رايح جاي ي طحلبي
4                                     تتمغط ومعها سداع
Name: tweet, dtype: object

* Pipeline for Representation & Classifier

* TfidfVectorizer Representation


Instead of just counting the frequency of each word, each word here is weighted using TF-IDF

$$W_{x, y} = tf_{x, y} \times log(\frac{N}{df_x})$$

In [ ]:
pipe1 = Pipeline([
    ('vectorizer', TfidfVectorizer(encoding='utf-8',)),        
    ('classifier',LinearSVC ())       
])

In [ ]:
pipe1.fit(x_train,y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()), ('classifier', LinearSVC())])

In [ ]:
y_pred_test1=pipe1.predict(x_test)
f'Accuracy is {accuracy_score(y_test, y_pred_test1) * 100:.3f} %'

'Accuracy is 78.898 %'

* countVectorizer Representation


Count the occurrences of words in the corpus.

In [ ]:
pipe2 = Pipeline([
    ('vectorizer', CountVectorizer(encoding='utf-8')),        
    ('classifier', MultinomialNB())       
])

In [ ]:
pipe2.fit(x_train,y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier', MultinomialNB())])

In [ ]:
y_pred_test2=pipe2.predict(x_test)
f'Accuracy is {accuracy_score(y_test, y_pred_test2) * 100:.3f} %'

'Accuracy is 78.238 %'